# Introduction à python-lxml

## Intérêts de Python
- Rapidité d'exécution
- Possibilité de 'brancher' le traitement à des outils puissants de TAL ou autres
- Un langage plus simple que XSLT
- Un traitement de corpus de documents *beaucoup* plus simple
  

## Limites de python-lxml

### Version de xpath limitée
Python-lxml ne supporte que la version 1.0 de XPath: un certain nombre de fonctions ne sont donc pas disponibles; elles sont cependant souvent remplaçables par des fonctions *ad-hoc* relativement faciles à produire en python.

### Non-récursivité
Python-lxml ne fonctionne pas selon les principes de récursivité propre au fonctionnement en *templates* de XSLT, qui vous sera présenté par Ariane Pinche. Par conséquent, ce langage est beaucoup moins adapté aux tâches de transformation profonde des documents XML, pour de la production d'éditions au format XML ou pdf par exemple.

## Comparaison

Pour résumer, XSLT sera adapté aux tâche de transformations de données structurées complexes en d'autres données structurées d'un niveau de complexité similaire. Python sera plus adapté aux tâches d'extraction et d'analyse des données. Il est bien entendu possible de combiner et d'alterner ces deux langages dans une même chaîne de traitement.


| **Tâche**                                                                     | **LXML** | **XSLT**       |
|-------------------------------------------------------------------------------|-----------------|----------------|
| Extraction simple de données                                                  | **Faisable**        | **Faisable**       |
| Traitement texte/image                                                        | **Faisable**        | Difficile      |
| Traitement automatique du langage (TAL): enrichissement, extraction d'entités | **Faisable**        | Très difficile |
| Production d'une édition complexe aux formats du web                        | Très difficile | **Faisable** |
| Production d'une édition critique en LaTeX                                    | Très difficile     | **Faisable** |
| Collation automatisée                                                         | **Faisable**        | Difficile      |


## Les espaces de nommage
Les espaces de nommage ou espaces de noms sont un concept propre au XML. Le XML a une double caractéristique quant au contrôle d'un document. Un document XML est en effet **bien formé** quand il respecte les règles fondamentales du XML (pas de chevauchement des éléments, attributs séparés par des espaces, etc). 

Le contrôle des données d'un document ne peut se limiter à la vérification de la conformité du document aux règles XML. Il doit aussi être valide, selon les règles édictées par un **schéma** qui prendra plusieurs formes (RNC, RNG, DTD). Le XML étant un format industriel, il est fréquent que les acteurs qui l'utilisent soient de grosses institutions et consortiums qui produisent des **spécifications standards**: nous pouvons citer ALTO, PAGE, SVG, DublinCore, et bien sûr la TEI ou la MEI. 

L'espace de nommage permet de préciser la spécification du XML que l'on a choisie. Un espace de nommage est représenté par URI (Uniform Ressource Identifier) et par un préfixe (qui sera présenté plus bas).

### "Mélanger" les spécifications
Il n'est pas interdit (et il est parfois utile) de "mélanger" les différentes spécifications XML au sein d'un document. Ainsi par exemple, un document TEI pourra contenir dans son `teiHeader` un ensemble d'éléments propres à la spécification de DublinCore:
```XML
<?xml version="1.0" encoding="UTF-8"?>
<TEI xmlns="http://www.tei-c.org/ns/1.0" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
   xmlns:dc="http://purl.org/dc/elements/1.1/">
   <teiHeader>      
      <fileDesc>
         <titleStmt>
            <title>Titre du document</title>
            <author>Auteur du document</author>
         </titleStmt>
         <publicationStmt>
            <publisher>Éditeur du document</publisher>
            <date>2023-10-01</date>
         </publicationStmt>
         <sourceDesc>
            <p>Source description</p>
         </sourceDesc>
      </fileDesc>
      <xenoData>
         <rdf:RDF>
            <rdf:Description rdf:about="http://www.worldcat.org/oclc/606621663">
               <dc:title>The description of a new world, called the blazing-world</dc:title>
               <dc:creator>The Duchess of Newcastle</dc:creator>
               <dc:date>1667</dc:date>
               <dc:identifier>British Library, 8407.h.10</dc:identifier>
               <dc:subject>utopian fiction</dc:subject>
            </rdf:Description>
         </rdf:RDF>
      </xenoData>
   </teiHeader>
   <text>
      <body>
         <p>Some text here.</p>
      </body>
   </text>
</TEI>

```
***

```XML
<?xml version="1.0" encoding="UTF-8"?>
<TEI xmlns="http://www.tei-c.org/ns/1.0" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
   xmlns:dc="http://purl.org/dc/elements/1.1/">
   <teiHeader>      
      <fileDesc>
         <titleStmt>
            <title>Titre du document</title>
            <author>Auteur du document</author>
         </titleStmt>
         <publicationStmt>
            <publisher>Éditeur du document</publisher>
            <date>2023-10-01</date>
         </publicationStmt>
         <sourceDesc>
            <p>Source description</p>
         </sourceDesc>
      </fileDesc>
      <xenoData>
         <rdf:RDF>
            <rdf:Description rdf:about="http://www.worldcat.org/oclc/606621663">
               <dc:title>The description of a new world, called the blazing-world</dc:title>
               <dc:creator>The Duchess of Newcastle</dc:creator>
               <dc:date>1667</dc:date>
               <dc:identifier>British Library, 8407.h.10</dc:identifier>
               <dc:subject>utopian fiction</dc:subject>
            </rdf:Description>
         </rdf:RDF>
      </xenoData>
   </teiHeader>
   <text>
      <body>
         <p>Some text here.</p>
      </body>
   </text>
</TEI>

```

Dans l'exemple ci-dessus, repris à partir des *Guidelines* de la TEI, on combine la TEI, RDF et DublinCore. Nous observons en particulier que nous avons deux éléments `title` qui n'appartiennent pas au même espace de nommage. Le premier appartient à la TEI, et le second à DublinCore. La nécessité  de pouvoir correctement et explicitement distinguer ces éléments est claire. 

On note que les éléments DublinCore et RDF sont identifiés par un code `rdf` ou `dc` suivi des deux points `:`. Ce code s'appelle un préfixe. Il doit être déclaré dans le noeud racine du document xml à l'aide d'attribut `xmlns` (*xml namespace*): 

`<TEI xmlns="http://www.tei-c.org/ns/1.0" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">`

Le premier espace de nom ne contient pas de préfixe: il s'agit de l'espace de nommage par défaut. Tous les éléments qui ne seront pas identifiés par un préfixe se rattacheront à cet espace de nommage. 

### Indiquer ou pas le préfixe?
Vous aurez le choix dans le traitement des sources XML d'expliciter ou pas l'espace de noms des éléments que vous voulez transformer (sauf dans le cas où vous auriez plusieurs espaces de noms). Je vous recommande d'être explicite et de toujours indiquer le préfixe.

### Dans lxml
La gestion des espaces de noms se fait à l'aide d'un dictionnaire dans lxml: pour chaque espace de nommage, la clé est le préfixe, et la valeur l'URI de l'espace de nommage.

In [1]:
tei_uri = "http://www.tei-c.org/ns/1.0"
dc_uri = "http://purl.org/dc/elements/1.1/"
namespaces_dict = {'tei': tei_uri,
                    'dc': dc_uri}

Nous n'utiliserons pas de sources contenant des éléments DublinCore, l'exemple est écrit ainsi pour vous donner une idée. Intéressons nous à la manipulation du document.

⚠ **ATTENTION** ⚠: Les premiers contacts avec les langages de transformation du XML peuvent être frustrants. Un adage de la communauté de la TEI est que si ça ne marche pas, c'est à cause de l'espace de nommage. Vérifiez-bien si tout est en règle de ce point de vue là quand il y a un problème!
***

## Parser le fichier
On va commencer par importer notre librairie.

In [2]:
import lxml.etree as etree

La méthode `parse()` nous dispense d'ouvrir le fichier avec `open()`:

In [3]:
fichier = 'fichier_xml_1.xml'
document_as_xml = etree.parse(fichier)
print(document_as_xml)

Nous obtenons un objet `lxml.etree._ElementTree`. Voyons les méthodes que l'on peut lui appliquer:

In [4]:
print(dir(document_as_xml))

['__class__', '__copy__', '__deepcopy__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__pyx_vtable__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '_setroot', 'docinfo', 'find', 'findall', 'findtext', 'getelementpath', 'getiterator', 'getpath', 'getroot', 'iter', 'iterfind', 'parse', 'parser', 'relaxng', 'write', 'write_c14n', 'xinclude', 'xmlschema', 'xpath', 'xslt']


Nous nous intéresserons presque exclusivement à la méthode `xpath()` ici, mais notez les méthodes `xinclude` (parser les éléments inclus via un élément `xi:include` -- encore un autre espace de noms! -- ou xslt pour appliquer une feuille de transformation xslt à l'objet sélectionné.

Voyons maintenant comment fonctionnent les différents noeuds XML dans lxml. Je me sers ici de la documentation officielle de lxml ([ici](https://lxml.de/tutorial.html)).
### Les éléments sont des listes
Notre arbre est représenté comme une liste de listes imbriquées:

In [5]:
root = document_as_xml.getroot()
print(root[0])

<Element {http://www.tei-c.org/ns/1.0}teiHeader at 0x7fc67e3a3180>


Le premier élément de notre arbre correspond à l'item d'index 0 de la liste: le `teiHeader`

In [6]:
print(root[1])

<Element {http://www.tei-c.org/ns/1.0}text at 0x7fc67f4a8640>


Le second élément correspond au `body`. On note bien que l'espace de noms est systématiquement explicité par lxml. Enfin, on peut aller plus loin dans la structure en interrogeant les sous-listes:

In [7]:
print(root[0][0])

<Element {http://www.tei-c.org/ns/1.0}fileDesc at 0x7fc67e3a3280>


Ici, on va trouver le premier élément du `teiHeader`.

### Les éléments portent leurs attributs comme des dictionnaires

## Sérialiser et enregistrer l'arbre
La sérialisation correspond à l'acte de sauvegarde d'une donnée à un format stable. Ici, plus simplement, il s'agit de l'enregistrement de notre document XML (qui a été désérialisé lors du *parsing*).

La méthode `etree.tostring()` permet de sérialiser un arbre importé: elle renvoie une chaîne de caractère en bytes qu'il va falloir convertir en chaîne. 

In [8]:
xml_as_string = etree.tostring(root, pretty_print=True).decode()
print(xml_as_string)

<TEI xmlns="http://www.tei-c.org/ns/1.0">
   <teiHeader>
      <fileDesc>
         <titleStmt>
            <title>Title</title>
         </titleStmt>
         <publicationStmt>
            <p>Publication Information</p>
         </publicationStmt>
         <sourceDesc>
            <p>Information about the source</p>
         </sourceDesc>
      </fileDesc>
         <encodingDesc>
            <!--...-->
            <metDecl pattern="((+|&#8203;-)+\|?/?)*">
               <metSym value="trochee" terminal="false">+-</metSym>
               <metSym value="iamb" terminal="false">-+</metSym>
               <metSym value="spondee" terminal="false">++</metSym>
               <metSym value="pyrrhic" terminal="false">--</metSym>
               <metSym value="amphibrach" terminal="false">-+-</metSym>
               <metSym value="anapaest" terminal="false">--+</metSym>
               <metSym value="+">metrical promimence</metSym>
               <metSym value="-">metrical non&#8203;-prominence</me

Nous pouvons maintenant enregistrer cette sérialisation:

In [9]:
with open("fichier_xml_1.copy.xml", "w") as output_xml:
    output_xml.write(xml_as_string)

## Naviguer dans l'arbre: la méthode `xpath()`

La méthode `xpath(requête, namespaces)` qui est appliquée sur un noeud XML (la racine ou autre) est la plus adéquate ici. Elle accepte des **requêtes XPath 1.0**. Le résultat produit est une liste (qui sera vide si l'élément cherché n'est pas présent dans le document) ou une valeur.

Attention aux espaces de noms avec le paramètre namespaces (au pluriel) qui doit renvoyer vers le dictionnaire recensant l'ensemble des espaces de noms du corpus. 

In [10]:
all_lg = root.xpath("descendant::tei:lg", namespaces=namespaces_dict)
print(all_lg)

[<Element {http://www.tei-c.org/ns/1.0}lg at 0x7fc67e3ab980>, <Element {http://www.tei-c.org/ns/1.0}lg at 0x7fc67e3ab9c0>, <Element {http://www.tei-c.org/ns/1.0}lg at 0x7fc67e3aba00>, <Element {http://www.tei-c.org/ns/1.0}lg at 0x7fc67e3aba40>, <Element {http://www.tei-c.org/ns/1.0}lg at 0x7fc67e3aba80>, <Element {http://www.tei-c.org/ns/1.0}lg at 0x7fc67e3abb00>]


⚠ **ATTENTION** ⚠: sélectionner un noeud sélectionne aussi la chaîne de caractère qui le suit directement (le *tail* en anglais) car elle est considérée comme rattachée à ce noeud. Cette caractéristique est propre à lxml et n'est pas partagée par les autres librairies de traitement de sources XML. Déplacer ou supprimer des noeuds peut donc mener à des conséquences imprévues et il faut l'avoir en tête à l'heure de travailler avec lxml.

In [11]:
caesura = root.xpath("descendant::tei:caesura", namespaces=namespaces_dict)[0]
print(etree.tostring(caesura, pretty_print=True).decode())

<caesura xmlns="http://www.tei-c.org/ns/1.0"/>yell



## Récupérer les noeuds textuels d'un noeud

## Créer un élément et l'insérer dans l'arbre

## Créer un attribut
La création d'attributs set fait grâce à la méthode `set('attribut', 'valeur')` appliquée à l'élément choisi. 

Imaginons que nous voulions indiquer que l'identification de la césure dans le dernier vers à l'aide de l'élément `caesura` est assumée de science certaine. Nous allons utiliser l'attribut `@cert` pour cela:

In [12]:
caesura = root.xpath("descendant::tei:lg[@type='couplet']/descendant::tei:caesura", namespaces=namespaces_dict)
print(caesura)

[<Element {http://www.tei-c.org/ns/1.0}caesura at 0x7fc67e3a1480>]


La méthode xpath renvoi (presque) toujours une liste même si elle ne contient qu'un élément: il faut donc veiller à le sélectionner explicitement.

In [13]:
caesura[0].set('cert', 'high')

Regardons le résultat en ciblant le couplet:

In [14]:
modified_lg = root.xpath("descendant::tei:lg[@type='couplet']", namespaces=namespaces_dict)[0]
print(etree.tostring(modified_lg, pretty_print=True).decode())

<lg xmlns="http://www.tei-c.org/ns/1.0" type="couplet">
                     <l>
                        <seg type="foot">But were</seg>
                        <seg type="foot"> some child</seg>
                        <seg type="foot"> of yours</seg>
                        <seg type="foot"> alive</seg>
                        <seg type="foot"> that time,</seg>
                     </l>
                     <l>
                        <seg type="foot">You should</seg>
                        <seg type="foot"> live twice-</seg>
                        <seg type="foot">in it,</seg>
                        <caesura cert="high"/>
                        <seg type="foot"> and in</seg>
                        <seg type="foot"> my rhyme. </seg>
                     </l>
                  </lg>
               



## Supprimer un élément
Il n'y a pas de manière directe de supprimer un élément. Il existe une méthode indirecte avec la méthode strip. Elle suppose d'identifier l'élément à supprimer, de sélectionner son parent, et de supprimer l'élément ciblé.  Supprimons notre élément `caesura`:

In [15]:
etree.strip_elements(caesura[0].getparent(), '{http://www.tei-c.org/ns/1.0}caesura', with_tail=False)

In [16]:
modified_lg = root.xpath("descendant::tei:lg[@type='couplet']", namespaces=namespaces_dict)[0]
print(etree.tostring(modified_lg, pretty_print=True).decode())

<lg xmlns="http://www.tei-c.org/ns/1.0" type="couplet">
                     <l>
                        <seg type="foot">But were</seg>
                        <seg type="foot"> some child</seg>
                        <seg type="foot"> of yours</seg>
                        <seg type="foot"> alive</seg>
                        <seg type="foot"> that time,</seg>
                     </l>
                     <l>
                        <seg type="foot">You should</seg>
                        <seg type="foot"> live twice-</seg>
                        <seg type="foot">in it,</seg>
                        
                        <seg type="foot"> and in</seg>
                        <seg type="foot"> my rhyme. </seg>
                     </l>
                  </lg>
               



In [17]:
## Document final
xml_as_string = etree.tostring(root, pretty_print=True).decode()
with open("fichier_xml_1_final.xml", "w") as output_xml:
    output_xml.write(xml_as_string)